In [ ]:
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

In [ ]:
printm()

In [ ]:
# Clone repo
!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor

In [ ]:
# Change directory
%cd keras_triplet_descriptor    


In [ ]:
# Download data
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip


In [ ]:
# Extract data
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

In [ ]:
import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose
from keras.layers import Input, UpSampling2D, concatenate  

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

In [ ]:
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

In [ ]:
hpatches_dir = './hpatches'
splits_path = 'splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   

In [ ]:
seqs_train = list(filter(lambda x: x.split('\\')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('\\')[-1] in split['test'], seqs)) 

In [ ]:
len(seqs)

In [ ]:
len(seqs_train)

In [ ]:
len(seqs_test)

## Models and loss

In [ ]:
def get_denoise_model(shape, do = 0, activate = 'selu'):
  
    inputs = Input(shape)

    ## Encoder starts
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    ## Bottleneck
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

    ## Now the decoder starts
    up3 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
    merge3 = concatenate([conv1,up3], axis = -1)
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)

    conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

    shallow_net = Model(inputs = inputs, outputs = conv4)

    return shallow_net



def get_descriptor_model(shape, activate= 'relu'):
  
    '''Architecture copies HardNet architecture'''

    init_weights = keras.initializers.he_normal()

    descriptor_model = Sequential()
    descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
    descriptor_model.add(BatchNormalization(axis = -1))
    descriptor_model.add(Activation(activate))

    descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
    descriptor_model.add(BatchNormalization(axis = -1))
    descriptor_model.add(Activation(activate))

    descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
    descriptor_model.add(BatchNormalization(axis = -1))
    descriptor_model.add(Activation(activate))

    descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
    descriptor_model.add(BatchNormalization(axis = -1))
    descriptor_model.add(Activation(activate))

    descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
    descriptor_model.add(BatchNormalization(axis = -1))
    descriptor_model.add(Activation(activate))

    descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
    descriptor_model.add(BatchNormalization(axis = -1))
    descriptor_model.add(Activation(activate))
    descriptor_model.add(Dropout(0.3))

    descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))

    # Final descriptor reshape
    descriptor_model.add(Reshape((128,)))

    return descriptor_model
  

def triplet_loss(x):
  
    output_dim = 128
    a, p, n = x
    _alpha = 1.0
    positive_distance = K.mean(K.square(a - p), axis=-1)
    negative_distance = K.mean(K.square(a - n), axis=-1)

    return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

## Denoising Image Patches


In [ ]:
from keras.layers import LeakyReLU
shape = (32, 32, 1)
denoise_model = keras.models.load_model('./denoise_base.h5')

## Vary Learning Rate

In [ ]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')

descriptor_model = get_descriptor_model( shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)
loss = Lambda(triplet_loss)([ea, ep, en])


sgd1 = keras.optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
sgd2 = keras.optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
sgd3 = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
sgd4 = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
sgd5 = keras.optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)

descriptor_model_trip_sgd1 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd2 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd3 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd4 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd5 = Model(inputs=[xa, xp, xn], outputs=loss)

descriptor_model_trip_sgd1.compile(loss='mean_absolute_error', optimizer=sgd1)
descriptor_model_trip_sgd2.compile(loss='mean_absolute_error', optimizer=sgd2)
descriptor_model_trip_sgd3.compile(loss='mean_absolute_error', optimizer=sgd3)
descriptor_model_trip_sgd4.compile(loss='mean_absolute_error', optimizer=sgd4)
descriptor_model_trip_sgd5.compile(loss='mean_absolute_error', optimizer=sgd5)

In [ ]:
### Descriptor loading and training
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=denoise_model, use_clean=False)

# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=10000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

In [ ]:
plot_triplet(training_generator)

In [ ]:
#epochs = 1
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer

#for e in range(epochs):
  
descriptor_history_sgd1 = descriptor_model_trip_sgd1.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd2 = descriptor_model_trip_sgd2.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd3 = descriptor_model_trip_sgd3.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd4 = descriptor_model_trip_sgd4.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd5 = descriptor_model_trip_sgd5.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

## Plot Losses 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_history(history, history2, history3, history4, history5, metric = None):
  # Plots the loss history of training and validation (if existing)
  # and a given metric
  
  if metric != None:
    fig, axes = plt.subplots(2,1, figsize=(8, 10))
    #axes[0].plot(history.history[metric])
    #axes[0].plot(history2.history[metric])
    #axes[0].plot(history3.history[metric])
    #axes[0].plot(history4.history[metric])
    #axes[0].plot(history5.history[metric])
    #axes[0].plot(history6.history[metric])
    try:
      #axes[0].plot(history.history['val_'+metric])
      #axes[0].plot(history2.history['val2_'+metric])
      #axes[0].plot(history3.history['val3_'+metric])
      axes[0].legend(['lr=1e-5', 'lr=1e-4', 'lr=1e-3', 'lr=1e-2', 'lr=1e-1'], loc='upper right')
    except:
      pass
    axes[0].set_title('MAE Vs. No of Epochs for Various Learning Rates')
    axes[0].set_ylabel('Mean Absolute Error')
    axes[0].set_xlabel('Epoch')
    fig.subplots_adjust(hspace=0.5)
    axes[1].plot(history.history['loss'])
    axes[1].plot(history2.history['loss'])
    axes[1].plot(history3.history['loss'])
    axes[1].plot(history4.history['loss'])
    axes[1].plot(history5.history['loss'])
    try:
      #axes[1].plot(history.history['val_loss'])
      axes[1].legend(['lr=1e-5', 'lr=1e-4', 'lr=1e-3', 'lr=1e-2', 'lr=1e-1'], loc='upper right')
    except:
      pass
    axes[1].set_title('MAE Vs. No of Epochs for Various Learning Rates')
    axes[1].set_ylabel('Mean Absolute Error')
    axes[1].set_xlabel('Epoch')
  else:
    plt.plot(history.history['loss'])
    try:
      plt.plot(history.history['val_loss'])
      plt.legend(['Train', 'Val'])
    except:
      pass
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
  
plot_history(descriptor_history_sgd1, descriptor_history_sgd2, descriptor_history_sgd3, descriptor_history_sgd4, descriptor_history_sgd5, 'mean_absolute_error')

In [ ]:
def plot_val_history(history, history2, history3, history4, history5, metric = None):
  # Plots the loss history of training and validation (if existing)
  # and a given metric
  
  if metric != None:
    fig, axes = plt.subplots(2,1, figsize=(8, 10))
    #axes[0].plot(history.history[metric])
    #axes[0].plot(history2.history[metric])
    #axes[0].plot(history3.history[metric])
    try:
      #axes[0].plot(history.history['val_'+metric])
      #axes[0].plot(history2.history['val_'+metric])
      #axes[0].plot(history3.history['val_'+metric])
      #axes[0].plot(history4.history['val_'+metric])
      #axes[0].plot(history5.history['val_'+metric])
      #axes[0].plot(history6.history['val_'+metric])
      axes[0].legend(['lr=1e-5', 'lr=1e-4', 'lr=1e-3', 'lr=1e-2', 'lr=1e-1'], loc='upper right')

    except:
      pass
    axes[0].set_title('Validation Loss Vs. No of Epochs for Various Learning Rates')
    axes[0].set_ylabel('Validation Loss')
    axes[0].set_xlabel('Epoch')
    fig.subplots_adjust(hspace=0.5)
    #axes[1].plot(history.history['loss'])
    #axes[1].plot(history2.history['loss'])
    #axes[1].plot(history3.history['loss'])
    try:
      axes[1].plot(history.history['val_loss'])
      axes[1].plot(history2.history['val_loss'])
      axes[1].plot(history3.history['val_loss'])
      axes[1].plot(history4.history['val_loss'])
      axes[1].plot(history5.history['val_loss'])
      #axes[1].plot(history6.history['val_loss'])
      axes[1].legend(['lr=1e-5', 'lr=1e-4', 'lr=1e-3', 'lr=1e-2', 'lr=1e-1'], loc='upper right')

    except:
      pass
    axes[1].set_title('Validation Loss Vs. No of Epochs for Various Learning Rates')
    axes[1].set_ylabel('Validation Loss')
    axes[1].set_xlabel('Epoch')
  else:
    plt.plot(history.history['loss'])
    try:
      plt.plot(history.history['val_loss'])
      plt.legend(['Train', 'Val'])
    except:
      pass
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
  
plot_val_history(descriptor_history_sgd1, descriptor_history_sgd2, descriptor_history_sgd3, descriptor_history_sgd4, descriptor_history_sgd5, 'mean_absolute_error')

## Vary Momentum

In [ ]:
sgd1 = keras.optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
sgd2 = keras.optimizers.SGD(lr=0.1, momentum=0.8, nesterov=True)
sgd3 = keras.optimizers.SGD(lr=0.1, momentum=0.7, nesterov=True)
sgd4 = keras.optimizers.SGD(lr=0.1, momentum=0.6, nesterov=True)
sgd5 = keras.optimizers.SGD(lr=0.1, momentum=0.5, nesterov=True)

descriptor_model_trip_sgd1 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd2 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd3 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd4 = Model(inputs=[xa, xp, xn], outputs=loss)
descriptor_model_trip_sgd5 = Model(inputs=[xa, xp, xn], outputs=loss)

descriptor_model_trip_sgd1.compile(loss='mean_absolute_error', optimizer=sgd1)
descriptor_model_trip_sgd2.compile(loss='mean_absolute_error', optimizer=sgd2)
descriptor_model_trip_sgd3.compile(loss='mean_absolute_error', optimizer=sgd3)
descriptor_model_trip_sgd4.compile(loss='mean_absolute_error', optimizer=sgd4)
descriptor_model_trip_sgd5.compile(loss='mean_absolute_error', optimizer=sgd5)

In [ ]:
#epochs = 1
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer

#for e in range(epochs):
  
descriptor_history_sgd1 = descriptor_model_trip_sgd1.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd2 = descriptor_model_trip_sgd2.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd3 = descriptor_model_trip_sgd3.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd4 = descriptor_model_trip_sgd4.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)

descriptor_history_sgd5 = descriptor_model_trip_sgd5.fit_generator(generator=training_generator, epochs=5, verbose=1, validation_data=val_generator)


### Saves optimizer and weights
#descriptor_model_trip.save('descriptor.h5') 
### Uploads files to external hosting
#!curl -F "file=@descriptor.h5" https://file.io

## Plot Losses

In [ ]:
def plot_history(history, history2, history3, history4, history5, metric = None):
  # Plots the loss history of training and validation (if existing)
  # and a given metric
  
  if metric != None:
    fig, axes = plt.subplots(2,1, figsize=(8, 10))
    #axes[0].plot(history.history[metric])
    #axes[0].plot(history2.history[metric])
    #axes[0].plot(history3.history[metric])
    #axes[0].plot(history4.history[metric])
    #axes[0].plot(history5.history[metric])
    
    try:
      #axes[0].plot(history.history['val_'+metric])
      #axes[0].plot(history2.history['val2_'+metric])
      #axes[0].plot(history3.history['val3_'+metric])
      axes[0].legend(['0.9', '0.8', '0.7', '0.6', '0.5'], loc='best')
    except:
      pass
    axes[0].set_title('MAE Vs. No of Epochs for Various Momentum Values')
    axes[0].set_ylabel('Mean Absolute Error')
    axes[0].set_xlabel('Epoch')
    fig.subplots_adjust(hspace=0.5)
    axes[1].plot(history.history['loss'])
    axes[1].plot(history2.history['loss'])
    axes[1].plot(history3.history['loss'])
    axes[1].plot(history4.history['loss'])
    axes[1].plot(history5.history['loss'])
    try:
      #axes[1].plot(history.history['val_loss'])
      axes[1].legend(['0.9', '0.8', '0.7', '0.6', '0.5'], loc='best')
    except:
      pass
    axes[1].set_title('MAE Vs. No of Epochs for Various Momentum Values')
    axes[1].set_ylabel('Mean Absolute Error')
    axes[1].set_xlabel('Epoch')
  else:
    plt.plot(history.history['loss'])
    try:
      plt.plot(history.history['val_loss'])
      plt.legend(['Train', 'Val'])
    except:
      pass
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
  
plot_history(descriptor_history_sgd1, descriptor_history_sgd2, descriptor_history_sgd3, descriptor_history_sgd4, descriptor_history_sgd5, 'mean_absolute_error')

In [ ]:
def plot_val_history(history, history2, history3, history4, history5, metric = None):
  # Plots the loss history of training and validation (if existing)
  # and a given metric
  
  if metric != None:
    fig, axes = plt.subplots(2,1, figsize=(8, 10))
    #axes[0].plot(history.history[metric])
    #axes[0].plot(history2.history[metric])
    #axes[0].plot(history3.history[metric])
    try:
      #axes[0].plot(history.history['val_'+metric])
      #axes[0].plot(history2.history['val_'+metric])
      #axes[0].plot(history3.history['val_'+metric])
      #axes[0].plot(history4.history['val_'+metric])
      #axes[0].plot(history5.history['val_'+metric])
      axes[0].legend(['0.9', '0.8', '0.7', '0.6', '0.5'], loc='best')
    except:
      pass
    axes[0].set_title('Validation Loss Vs. No of Epochs for for Various Momentum Values')
    axes[0].set_ylabel('Validation Loss')
    axes[0].set_xlabel('Epoch')
    fig.subplots_adjust(hspace=0.5)
    #axes[1].plot(history.history['loss'])
    #axes[1].plot(history2.history['loss'])
    #axes[1].plot(history3.history['loss'])
    try:
      axes[1].plot(history.history['val_loss'])
      axes[1].plot(history2.history['val_loss'])
      axes[1].plot(history3.history['val_loss'])
      axes[1].plot(history4.history['val_loss'])
      axes[1].plot(history5.history['val_loss'])
      axes[1].legend(['0.9', '0.8', '0.7', '0.6', '0.5'], loc='best')
    except:
      pass
    axes[1].set_title('Validation Loss Vs. No of Epochs for Various Momentum Values')
    axes[1].set_ylabel('Validation Loss')
    axes[1].set_xlabel('Epoch')
  else:
    plt.plot(history.history['loss'])
    try:
      plt.plot(history.history['val_loss'])
      plt.legend(['Train', 'Val'])
    except:
      pass
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    
plot_val_history(descriptor_history_sgd1, descriptor_history_sgd2, descriptor_history_sgd3, descriptor_history_sgd4, descriptor_history_sgd5, 'mean_absolute_error')

## Save Baseline Model

In [ ]:
sgd1 = keras.optimizers.SGD(lr=0.1, momentum=0.7, nesterov=True)

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)

descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd1)

In [ ]:
#epochs = 1
### As with the denoising model, we use a loop to save for each epoch 
## #the weights in an external website in case colab stops. 
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)

### If you have a model saved from a previous training session
### Load it in the next line
# descriptor_model_trip.set_weights(keras.models.load_model('./descriptor.h5').get_weights())
# descriptor_model_trip.optimizer = keras.models.load_model('./descriptor.h5').optimizer

#for e in range(epochs):
  
descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=20, verbose=1, validation_data=val_generator)
descriptor_model_trip.save('descriptor_base.h5') 

### Saves optimizer and weights
#descriptor_model_trip.save('descriptor.h5') 
### Uploads files to external hosting
#!curl -F "file=@descriptor.h5" https://file.io

## Test mAP on Baseline Approach

generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

## Verification

In [ ]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

## Matching

In [ ]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

## Retrieval

In [ ]:
!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval